In [ ]:
import json
import http.client
from urllib.parse import urlencode
import logging
from datetime import datetime,timedelta,date
import pandas as pd
import ta
import os
from klse_scrapper import *
from session_handler import *
import requests
import yfinance as yf
import urllib.parse
import zipfile
from workalendar.asia import Malaysia
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
from ggplot import *
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks as cf
cf.go_offline(connected=True)
init_notebook_mode(connected=True)
from loess.loess_1d import loess_1d
import plotly.figure_factory as ff
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
import plotly.offline as py

In [ ]:
def lambda_handler(event):
#Demonstrates a simple HTTP request from Lambda
    logger.debug(event)
    symbol=event['counter']
    resolution=event['resolution']
    #end_time=int(datetime.now().timestamp())
    #start_time=int((datetime.now() - timedelta(hours=24)).timestamp())
    start_time=event['from']
    end_time=event['to']
    logger.debug("Start time:" + str(start_time) + " End time:" + str(end_time))
    headers = {"Content-type": "application/json"}
    payload = {"symbol":symbol,"resolution":resolution,"from":start_time,"to":end_time}
    logger.debug(urlencode(payload))
    conn = http.client.HTTPSConnection("www.klsescreener.com")
    conn.request("GET", "/v2/trading_view/history?"+urlencode(payload))
    response = conn.getresponse()
    res = response.read().decode() #load data into a dict of objects, posts
    if(json.loads(res)['s']=='ok'):
        return pd.read_json(res)
    else:
        return {}

In [ ]:
def lambda_handler_yf(event):
#Demonstrates a simple HTTP request from Lambda
    logger.debug(event)
    symbol=event['counter']
    resolution=event['resolution']
    #end_time=int(datetime.now().timestamp())
    #start_time=int((datetime.now() - timedelta(hours=24)).timestamp())
    start_time=datetime.fromtimestamp(event['from']).strftime("%Y-%m-%d")
    end_time=datetime.fromtimestamp(event['to']).strftime("%Y-%m-%d")
    ticker = yf.Ticker(str(event['counter'])+'.KL')
    hist = ticker.history(start=start_time, end=end_time,interval=event['resolution'])
    return hist

In [ ]:
def distribute_requests(event):
    #split query time if longer than 1 day
    events=[]
    if(event['resolution']=='1m'):
        diff=event['to']-event['from']
        if(diff>86400):
            while(event['to']>event['from']):                
                event_copy=event.copy()
                event_copy['from']=event_copy['to']-86400
                event['to']=event['to']-86400
                events.append(event_copy)
    return events

In [ ]:
def plot_timeseries(df):
    df['t']=pd.to_datetime(df['t'],unit='s')
    df=df.set_index('t').drop(['s'],axis=1)
    df.iplot()

In [ ]:
def plot_noindex(df):
    df['t']=pd.to_datetime(df['t'],unit='s')
    df=df.drop(['s','t'],axis=1)
    df.iplot()

In [ ]:
def build_event_lists(lists_,from_,to_,resolution_='1m',csv_=1):
    events=[]
    for list_ in lists_:
        event={"category":list_['cat'],"symbol":list_['symbol'],"counter":list_['code'],"resolution":resolution_, "from":from_,"to":to_, "write_csv":csv_}
        events.append(event)
    return events

In [ ]:
def download_url(session, url, save_path, chunk_size=128):
    r = session.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [ ]:
def crawl_data(events):
    for event_ in events:
        events__ = distribute_requests(event_)
        for event__ in events__:
            df_=lambda_handler_yf(event__)
            name_=event__['symbol']+'_'+str(event__['counter'])
            dir_=os.path.join('history','price')
            dir_=os.path.join(dir_,event_['category'])
            dir_=os.path.join(dir_,name_)
            if len(df_)>0:        
                if not os.path.exists(dir_): os.makedirs(dir_)
                if(event__['write_csv']):
                    csv_name=name_+'_'+datetime.fromtimestamp(event__['from']).strftime("%Y-%m-%d")
                    df_.to_csv(os.path.join(dir_,csv_name+'.csv'))

In [ ]:
def crawl_quote_movements(event):
    date_=datetime.fromtimestamp(event['to'])
    cal = Malaysia()
    if(cal.is_working_day(date(date_.year,date_.month,date_.day))):
        date_=date_.strftime("%Y%m%d")
        quotes_url='http://www.shareinvestor.com/prices/price_download_zip_file.zip'
        params = {'type': 'quotemovements', 'counter': event['counter']+'.MY', 'date':date_}
        params=urllib.parse.urlencode(params)
        quotes_url=quotes_url+'?'+params
        session=read_session()
        dir_='history/quote_movements/'
        dir_=os.path.join(dir_,event['category'])
        name=event['symbol']+'_'+event['counter']
        dir_=os.path.join(dir_,name)
        if not os.path.exists(dir_): os.makedirs(dir_)
        save_path=os.path.join(dir_,name)+'.zip'
        try:
            download_url(session, quotes_url, save_path)
            with zipfile.ZipFile(save_path, 'r') as zip_ref:
                zip_ref.extractall(dir_)
            zip_ref.close()
            os.remove(save_path)
        except Exception as e:
            print(e)

In [ ]:
#resolution= [1m, 5m, 1d, 1w]
def mine_price_data(board,category,resolution,from_,to_):
    df=get_board_category_listings(board,category)
    lists_=df[['symbol','code','cat']].to_dict('records')
    events=build_event_lists(lists_=lists_,resolution_=resolution,from_=from_,to_=to_)
    crawl_data(events)

In [ ]:
def mine_quote_movements(board,category,from_,to_):
    df=get_board_category_listings(board,category)
    lists_=df[['symbol','code','cat']].to_dict('records')
    events=build_event_lists(lists_=lists_,from_=from_,to_=to_)
    for event in events:
        events_=distribute_requests(event)
        for event_ in events_:
            crawl_quote_movements(event_)

In [ ]:
#test case for mine_quote_movement
generate_session()
mine_quote_movements('Main Market','Industrial Products & Services',1591459200,1591977600)

In [ ]:
#test case for mine price data
mine_price_data('Main Market','Industrial Products & Services','1m',1591459200,1591977600)

In [ ]:
res=lambda_handler(event)
df=pd.read_json(res)
#plot_timeseries(df)
df['t']=pd.to_datetime(df['t'],unit='s')
df=df.set_index('t').drop(['s'],axis=1)
ta_indicators=ta.trend.IchimokuIndicator(high=df['h'],low=df['l'],visual=True,fillna=True)
df['ichimoku_a']=ta_indicators.ichimoku_a()
df['ichimoku_b']=ta_indicators.ichimoku_b()
df['ichimoku_base_line']=ta_indicators.ichimoku_base_line()
df['ichimoku_conversion_line']=ta_indicators.ichimoku_conversion_line()
#df[['c','ichimoku_a','ichimoku_b','ichimoku_base_line','ichimoku_conversion_line']].iplot()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['ichimoku_a'].values,
    fill=None,
    mode='lines',
    line_color='blue',
    ))
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['ichimoku_b'].values,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='green'))

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['ichimoku_conversion_line'].values,
    mode='lines', line_color='orange'))

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['ichimoku_base_line'].values,
    mode='lines', line_color='purple'))

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['c'].values,
    mode='lines', line_color='red'))

fig.show()

In [ ]:
df=get_listings()